## 事前準備

In [ ]:
!sudo apt update
!sudo apt -y install ffmpeg

In [3]:
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

## 無音動画 & 静止画像作成

**[注意]** `gs://[PROJECT_ID]-handson/mp4_original/` 以下に、オリジナルの動画ファイル（mp4）を事前にアップロードしておきます。

In [1]:
[PROJECT_ID] = !gcloud config get-value project
BUCKET = f'gs://{PROJECT_ID}-handson'

In [2]:
import glob
from PIL import Image
from tqdm.notebook import tqdm

def process_file(target):
    gsutil_opt = '-o GSUtil:parallel_composite_upload_threshold=150M'
    ffmpeg_opt = '-loglevel error -v error -stats'
    filename = target.split('/')[-1]
    basename = filename.rstrip('.mp4')    
    filename2 = f'{BUCKET}/mp4/s_' + filename
    filename3 = f'{BUCKET}/mp4/n_' + filename
    image_dir = f'{BUCKET}/image/'
    
    !rm -f tmpfile1.mp4 tmpfile2.mp4 tmpfile3.mp4
    !gsutil {gsutil_opt} cp "{target}" tmpfile1.mp4
    
    # サイズ圧縮動画
    !ffmpeg {ffmpeg_opt} -i tmpfile1.mp4 -crf 31 tmpfile2.mp4
    !gsutil {gsutil_opt} cp tmpfile2.mp4 "{filename2}"

    # 無音動画
    !ffmpeg {ffmpeg_opt} -i tmpfile2.mp4 -vcodec copy -an tmpfile3.mp4
    !gsutil {gsutil_opt} cp tmpfile3.mp4 "{filename3}"    
    
    # 静止画像
    !mkdir -p "{basename}"
    !ffmpeg {ffmpeg_opt} -i tmpfile2.mp4 -r 1 "{basename}/capture%04d.png"
    
    # png を jpg に変換
    width, _ = Image.open(f'{basename}/capture0001.png').size
    width = min(width, 1280)
    file_list = glob.glob(f'{basename}/capture*.png')
    for img_path in tqdm(file_list):
        img_path2 = img_path.rstrip('.png') + '.jpg'
        !mv -f "{img_path}" tmp.png
        !ffmpeg {ffmpeg_opt} -i tmp.png -vf scale="{width}:-1" -q 2 "{img_path2}" 1>/dev/null 2>&1
    !gsutil -m {gsutil_opt} cp -r "{basename}" "{image_dir}/"
    !rm -f tmpfile1.mp4 tmpfile2.mp4 tmpfile3.mp4 tmp.png

In [3]:
targets = !gsutil ls {BUCKET}/mp4_original/*.mp4
targets

['gs://[PROJECT_ID]-handson/mp4_original/Google Cloud Next Tokyo ’24 - Innovators Hive の Day 0 に潜入！ -.mp4',
 'gs://[PROJECT_ID]-handson/mp4_original/千葉県印西市にデータセンターを開設.mp4',
 'gs://[PROJECT_ID]-handson/mp4_original/大規模言語モデルを支える分散学習インフラ Pathways.mp4']

In [ ]:
for target in targets:
    process_file(target)

In [5]:
!gsutil ls {BUCKET}/mp4/

gs://[PROJECT_ID]-handson/mp4/n_Google Cloud Next Tokyo ’24 - Innovators Hive の Day 0 に潜入！ -.mp4
gs://[PROJECT_ID]-handson/mp4/n_千葉県印西市にデータセンターを開設.mp4
gs://[PROJECT_ID]-handson/mp4/n_大規模言語モデルを支える分散学習インフラ Pathways.mp4
gs://[PROJECT_ID]-handson/mp4/s_Google Cloud Next Tokyo ’24 - Innovators Hive の Day 0 に潜入！ -.mp4
gs://[PROJECT_ID]-handson/mp4/s_千葉県印西市にデータセンターを開設.mp4
gs://[PROJECT_ID]-handson/mp4/s_大規模言語モデルを支える分散学習インフラ Pathways.mp4


In [6]:
!gsutil ls {BUCKET}/image/

gs://[PROJECT_ID]-handson/image/Google Cloud Next Tokyo ’24 - Innovators Hive の Day 0 に潜入！ -/
gs://[PROJECT_ID]-handson/image/千葉県印西市にデータセンターを開設/
gs://[PROJECT_ID]-handson/image/大規模言語モデルを支える分散学習インフラ Pathways/
